In [1]:
import qutip as qt
import numpy as np
from math import cos, sin, acos, pow
pi = acos(0)*2

In [2]:
H = (cos(pi/8)*qt.basis(2,0) + sin(pi/8)*qt.basis(2,1)).unit()

In [3]:
H

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.92387953]
 [ 0.38268343]]

Using a result from [Aaronson & Gottesman](https://arxiv.org/abs/quant-ph/0406196) we can calculate the number of n-qubit pure stabiliser states

In [4]:
def n_stab(n):
    res = pow(2.,n)
    for i in range(n):
        res *= (pow(2.,n-i)+1)
    return res
n_stab(2)

60.0

In [61]:
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
I = qt.qeye(2)
base_elements = [I, X, Y, Z, -1*X, -1*Y, -1*Z]
combos = []
for i in range(len(base_elements)):
    for j in range(len(base_elements)):
        res = qt.tensor(base_elements[i], base_elements[j])
        if any([el == res for el in combos]):
            continue
        else:
            combos.append(res)
len(combos)

31

In [62]:
from qutip.operators import commutator

def test_equality(g1,g2):
    if g1[0] in g2 and g1[1] in g2 and g1[2] in g2:
        return True
    else:
        return False
combos = combos[1:]

In [63]:
null = qt.Qobj([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]], dims=[[2,2],[2,2]])

In [64]:
n_found = 0
groups = []
while n_found < 60:
    p = combos.pop(0)
    for p2 in combos:
        if commutator(p,p2) == null:
            candidate_group = [p, p2, p*p2]
            if len(groups) == 0:
                if any([test_equality(candidate_group, group) for group in groups]):
                    continue
                else:
                    groups.append(candidate_group)
                    n_found += 1
            else:
                groups.append(candidate_group)
                n_found += 1

In [74]:
eqs = [test_equality(groups[0],groups[i]) for i in range(len(groups))]

In [76]:
def test_mutual_c(group):
    for i in range(len(group)):
        if not all([commutator(group[i], group[j]) for j in range(len(group))]):
            print("Something went wrong")